In [47]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.161  Python-3.9.17 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
Setup complete  (32 CPUs, 31.8 GB RAM, 164.2/953.1 GB disk)


In [48]:
import os
import random
import cv2
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

# MODEL = 'yolov8s.pt'  # inference time 4.0 ms on 640 x 384 x 3, 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs
# MODEL = 'yolov8m.pt'  # inference time 4.0 ms on 640 x 384 x 3, 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs
# MODEL = 'yolov8l.pt'  # inference time 6.0 ms on 640 x 384 x 3, 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs
MODEL = 'yolov8x.pt'  # inference time 10.0 ms on 640 x 384 x 3, 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

model = YOLO(MODEL)
model.fuse()

CLASS_NAMES_DICT = model.model.names

# class_ids of interest - person, car, motorcycle, bus, truck, horses, sports ball
# [0, 2, 3, 5, 7, 17, 32]
CLASS_ID = [0] # person  

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [40]:
print(CLASS_NAMES_DICT.values())

dict_values(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'])


### file name 변경 테스트 코드는 실행을 스킵할 것

In [ ]:
# file name 변경 테스트 코드
import os
import shutil
ifile_path = "C:/Users/User2/source/pcounter/20230828-학습용데이터셋구성/filename_test/inputs"
ofile_path = "C:/Users/User2/source/pcounter/20230828-학습용데이터셋구성/filename_test/results"
filenames = os.listdir(ifile_path)
# ofilenames = []

i = 1

for filename in filenames:
    src_file = os.path.join(ifile_path, filename)
    pat, ext = os.path.splitext(filename)
    print(f'filename: {filename} ----- [body: \'{pat}\', extension: \'{ext}\']')
    # ofilenames.append(os.path.join(ofile_path, 'new_readme'+f'{i:03d}'+ext))      
    dst_file = os.path.join(ofile_path, 'new_readme'+f'{i:03d}'+ext)    
    # dst_file = os.path.join(ofile_path, pat+'.jpg')    
    print(f'renamed: {dst_file}\n')
    shutil.copyfile(src_file, dst_file)
    i += 1
# print(ofilenames)


In [ ]:
# Yolo 라벨링 값의 포맷
# class_ID center_x center_y box_width box_height

# class_ID = {0: con_eq, 1: worker}
# center_x = box_center_x / image_width
# center_y = box_center_y / image_height
# box_w = det_width / image_width
# box_h = det_height / image_height

SOURCE_DATA_PATH = 'C:/Users/User2/source/pcounter/data/people/test/images'
TARGET_LABEL_PATH = 'C:/Users/User2/source/pcounter/data/people/test/labels'

image_files = os.listdir(SOURCE_DATA_PATH)

for i in range(len(image_files)):
    image_file = random.choice(image_files)
    image_file_ful_path = f'{SOURCE_DATA_PATH}/'+image_file
    pat, ext = os.path.splitext(image_file)
    print(image_file_ful_path)

    img = cv2.imread(image_file_ful_path, cv2.IMREAD_COLOR)
    img_h, img_w, img_c = img.shape 
    
    results = model(img) # inference
    
    xyxy0=results[0].boxes.xyxy.cpu().numpy()
    confidence0=results[0].boxes.conf.cpu().numpy()
    class_id0=results[0].boxes.cls.cpu().numpy().astype(int)

    label_file = open(f'{TARGET_LABEL_PATH}/'+pat+'.txt', 'w')

    num_objects = len(class_id0)
    for i in range(num_objects):     
        if confidence0[i] > 0.3:    
            class_id = class_id0[i]
            fx1 = xyxy0[i][0]
            fy1 = xyxy0[i][1]
            fx2 = xyxy0[i][2]
            fy2 = xyxy0[i][3]

            box_center_x = (fx1 + fx2)/2
            box_center_y = (fy1 + fy2)/2
            center_x = box_center_x / img_w
            center_y = box_center_y / img_h
            det_w = fx2-fx1
            det_h = fy2-fy1
            box_w = det_w / img_w
            box_h = det_h / img_h


            label_file.write(f'{class_id} {center_x:.6f} {center_y:.6f} {box_w:.6f} {box_h:.6f}\n')
            # label_file.write(f'{class_id} {fx1:.6f} {fy1:.6f} {fw:.6f} {fh:.6f}\n')            

    label_file.close()

In [49]:
# Yolo 라벨링 값의 포맷
# class_ID center_x center_y box_width box_height

# class_ID = {0: con_eq, 1: worker}
# center_x = box_center_x / image_width
# center_y = box_center_y / image_height
# box_w = det_width / image_width
# box_h = det_height / image_height

SOURCE_DATA_PATH = 'C:/Users/User2/source/pcounter/data/people/train/images'
TARGET_LABEL_PATH = 'C:/Users/User2/source/pcounter/data/people/train/labels'

image_files = os.listdir(SOURCE_DATA_PATH)

for i in range(len(image_files)):
    image_file = random.choice(image_files)
    image_file_ful_path = f'{SOURCE_DATA_PATH}/'+image_file
    pat, ext = os.path.splitext(image_file)
    print(image_file_ful_path)

    img = cv2.imread(image_file_ful_path, cv2.IMREAD_COLOR)
    img_h, img_w, img_c = img.shape 
    
    results = model(img) # inference
    
    xyxy0=results[0].boxes.xyxy.cpu().numpy()
    confidence0=results[0].boxes.conf.cpu().numpy()
    class_id0=results[0].boxes.cls.cpu().numpy().astype(int)

    label_file = open(f'{TARGET_LABEL_PATH}/'+pat+'.txt', 'w')

    num_objects = len(class_id0)
    for i in range(num_objects):     
        if confidence0[i] > 0.3:    
            class_id = class_id0[i]
            fx1 = xyxy0[i][0]
            fy1 = xyxy0[i][1]
            fx2 = xyxy0[i][2]
            fy2 = xyxy0[i][3]

            box_center_x = (fx1 + fx2)/2
            box_center_y = (fy1 + fy2)/2
            center_x = box_center_x / img_w
            center_y = box_center_y / img_h
            det_w = fx2-fx1
            det_h = fy2-fy1
            box_w = det_w / img_w
            box_h = det_h / img_h


            label_file.write(f'{class_id} {center_x:.6f} {center_y:.6f} {box_w:.6f} {box_h:.6f}\n')
            # label_file.write(f'{class_id} {fx1:.6f} {fy1:.6f} {fw:.6f} {fh:.6f}\n')            

    label_file.close()

C:/Users/User2/source/pcounter/data/people/train/images/p_train1736.png


0: 384x640 3 persons, 1 umbrella, 1 suitcase, 2 tennis rackets, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 bench, 1 bird, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 cat, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 bench, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 bird, 1 tennis racket,

C:/Users/User2/source/pcounter/data/people/train/images/p_train0505.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2904.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2429.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0956.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0510.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2015.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1352.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0208.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1948.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0095.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3547.png


Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bench, 1 dog, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 2 birds, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 cat, 1 suitcase, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 suitcase, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 bottle, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 bench, 1 cake, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postproces

C:/Users/User2/source/pcounter/data/people/train/images/p_train1406.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1646.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1015.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0502.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0262.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2380.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1310.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1271.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2331.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0061.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2249.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0467.png


...

C:/Users/User2/source/pcounter/data/people/train/images/p_train0531.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0759.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2703.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1939.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3632.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0637.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3358.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1158.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1770.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0173.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2907.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1420.png


Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 bench, 1 bird, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 suitcase, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 backpack, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 suitcase, 1 bottle, 9.0ms
Speed: 0.6ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 cat, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 8.3ms
Speed: 1.0ms preprocess, 8.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 bird, 1 skateboard, 9.0ms
Speed: 1.0ms preprocess, 9.0ms infere

C:/Users/User2/source/pcounter/data/people/train/images/p_train2200.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0422.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1071.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2716.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0689.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3003.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2343.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3220.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3011.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1619.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train1563.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2150.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train2029.png


Speed: 1.3ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 bench, 8.3ms
Speed: 1.0ms preprocess, 8.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 backpack, 8.7ms
Speed: 0.0ms preprocess, 8.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


C:/Users/User2/source/pcounter/data/people/train/images/p_train3149.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train3628.png
C:/Users/User2/source/pcounter/data/people/train/images/p_train0569.png


In [50]:
# Yolo 라벨링 값의 포맷
# class_ID center_x center_y box_width box_height

# class_ID = {0: con_eq, 1: worker}
# center_x = box_center_x / image_width
# center_y = box_center_y / image_height
# box_w = det_width / image_width
# box_h = det_height / image_height

SOURCE_DATA_PATH = 'C:/Users/User2/source/pcounter/data/people/valid/images'
TARGET_LABEL_PATH = 'C:/Users/User2/source/pcounter/data/people/valid/labels'

image_files = os.listdir(SOURCE_DATA_PATH)

for i in range(len(image_files)):
    image_file = random.choice(image_files)
    image_file_ful_path = f'{SOURCE_DATA_PATH}/'+image_file
    pat, ext = os.path.splitext(image_file)
    print(image_file_ful_path)

    img = cv2.imread(image_file_ful_path, cv2.IMREAD_COLOR)
    img_h, img_w, img_c = img.shape 
    
    results = model(img) # inference
    
    xyxy0=results[0].boxes.xyxy.cpu().numpy()
    confidence0=results[0].boxes.conf.cpu().numpy()
    class_id0=results[0].boxes.cls.cpu().numpy().astype(int)

    label_file = open(f'{TARGET_LABEL_PATH}/'+pat+'.txt', 'w')

    num_objects = len(class_id0)
    for i in range(num_objects):     
        if confidence0[i] > 0.3:    
            class_id = class_id0[i]
            fx1 = xyxy0[i][0]
            fy1 = xyxy0[i][1]
            fx2 = xyxy0[i][2]
            fy2 = xyxy0[i][3]

            box_center_x = (fx1 + fx2)/2
            box_center_y = (fy1 + fy2)/2
            center_x = box_center_x / img_w
            center_y = box_center_y / img_h
            det_w = fx2-fx1
            det_h = fy2-fy1
            box_w = det_w / img_w
            box_h = det_h / img_h


            label_file.write(f'{class_id} {center_x:.6f} {center_y:.6f} {box_w:.6f} {box_h:.6f}\n')
            # label_file.write(f'{class_id} {fx1:.6f} {fy1:.6f} {fw:.6f} {fh:.6f}\n')            

    label_file.close()


0: 384x640 4 persons, 1 bench, 164.5ms
Speed: 4.3ms preprocess, 164.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)



C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0133.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0159.png


0: 384x640 4 persons, 2 birds, 1 suitcase, 68.9ms
Speed: 2.0ms preprocess, 68.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 bench, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 handbag, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 skateboard, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 birds, 1 suitcase, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 backpack, 1 snowboard, 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 person

C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0133.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0324.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0051.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0331.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0170.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0126.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0318.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0240.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0103.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0341.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0298.png


Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 truck, 1 skateboard, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 bench, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 cat, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 truck, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bench, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (

...

C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0452.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0110.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0011.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0132.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0248.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0252.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0033.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0027.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0327.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0125.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0412.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0204.png


Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 suitcase, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 suitcase, 1 skateboard, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 boat, 1 suitcase, 1 sports ball, 1 skateboard, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 suitcase, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 bench, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 suitcase, 9.0ms
Speed

C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0195.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0434.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0316.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0217.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0295.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0173.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0204.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0311.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0319.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0209.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0041.png



0: 384x640 3 persons, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 suitcase, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0273.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0127.png
C:/Users/User2/source/pcounter/data/people/valid/images/p_valid0192.png
